In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
import tensorflow as tf

In [3]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [4]:
import mdtraj, nglview

In [5]:
rcParams.update({'font.size': 16})

In [6]:
# Switch AUTORELOAD ON. Disable this when in production mode!
%load_ext autoreload
%autoreload 2

In [7]:
from deep_boltzmann.networks.invertible import EnergyInvNet, invnet
from deep_boltzmann.networks.plot import test_xz_projection, test_generate_x
from deep_boltzmann.util import count_transitions
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite
from deep_boltzmann.networks.training import MLTrainer, FlexibleTrainer
from deep_boltzmann.util import save_obj, load_obj

BPTI
-----

In [1]:
import numpy as np
from matplotlib import rcParams
import keras
import tensorflow as tf

from keras.utils.vis_utils import model_to_dot

from deep_boltzmann.networks.invertible import invnet
from deep_boltzmann.models.openmm import OpenMMEnergy
from deep_boltzmann import openmmutils
from deep_boltzmann.networks.invertible import EnergyInvNet
import mdtraj as md
from simtk import openmm, unit

Using TensorFlow backend.


In [3]:
# setup BPTI
INTEGRATOR_ARGS = (300*unit.kelvin,
                   1.0/unit.picoseconds,
                    2.0*unit.femtoseconds)

data_sets=[]
pdb = openmm.app.PDBFile('./bpti/bpti.pdb')
forcefield = openmm.app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')

system = forcefield.createSystem(pdb.topology,
    nonbondedMethod=openmm.app.CutoffNonPeriodic, nonbondedCutoff=1.0*unit.nanometers,
    constraints=None, rigidWater=True)
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds,
    2.0*unit.femtoseconds)

In [4]:
simulation = openmm.app.Simulation(pdb.topology, system, integrator)

In [5]:
bpti_omm_energy = OpenMMEnergy(system,
                               openmm.LangevinIntegrator,
                               unit.nanometers,
                               n_atoms=md.Topology().from_openmm(simulation.topology).n_atoms,
                               openmm_integrator_args=INTEGRATOR_ARGS
                              )

mdtraj_topology = md.Topology().from_openmm(pdb.topology)

cartesian = ['CA', 'C', 'N']
cart = mdtraj_topology.select(' '.join(["name " + s for s in cartesian]))
from deep_boltzmann.models.proteins import mdtraj2Z
Z_ = np.array(mdtraj2Z(mdtraj_topology))